In [4]:
import ee
ee.Initialize(project='pyregence-ee')
import datetime
import scripts.analysis_functions as af
import scripts.utils as utils
#individual fire export uses geetools
import geetools
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Run BS Mapper for a particular year

#### Simulation runs

In [62]:
# For running a year of fires with a SIMULATED run date (as if run on this date)
# REQUIRES USER INPUT/SELECTION, see starred section below. 
#  Uses specialized subfunctions that are variants of original functions
#  Developed using MTBS fires, would need modification for NIFC (most likely, unless field names matched)

# Last edited 2023-09-19 by dnekorchuk

#***
# USER: 
# 1. Set data import, align year and import dataset
# 2. Scroll down and set export flags
# 3. Select simulation date (by index)
# 4. Set export flags for various results
# 5. Change out folder if needed
# 6. Note that fire filter date is set to the same for all sims (in a year), change if needed.
#  Developed with MTBS, will likely need modifications for NIFC
#***

#### IMPORT DATA ******************************************
#One set of yearly fires, no need for remove_recent() or year/ac filtering, that was already done upon data fetch

## OPTION 1a: MTBS option 2020
#align year with fires import below
yr = '2020' 
fires = ee.FeatureCollection("projects/pyregence-ee/assets/conus/mtbs/mtbs_fires_2020_20230406")
data_origin = 'mtbs'
asset_folder = 'projects/pyregence-ee/assets/fires_bs_tool/dmn_testing'

# ## OPTION 1b: MTBS option 2021
# #align year with fires import below
# yr = '2021' 
# fires = ee.FeatureCollection("projects/pyregence-ee/assets/conus/mtbs/mtbs_fires_2021_20230810")
# data_origin = 'mtbs'
# asset_folder = 'projects/pyregence-ee/assets/fires_bs_tool/dmn_testing'

# ## OPTION 1c: MTBS option 2019
# #align year with fires import below
# yr = '2019' 
# fires = ee.FeatureCollection("projects/pyregence-ee/assets/conus/mtbs/mtbs_fires_2019_20230920") 
# data_origin = 'mtbs'
# asset_folder = 'projects/pyregence-ee/assets/fires_bs_tool/dmn_testing'

# ## OPTION 1d: MTBS option 2017
# #align year with fires import below
# yr = '2017' 
# fires = ee.FeatureCollection("projects/pyregence-ee/assets/conus/mtbs/mtbs_fires_2017_20230920") 
# data_origin = 'mtbs'
# asset_folder = 'projects/pyregence-ee/assets/fires_bs_tool/dmn_testing'


# ## OPTION 2: Test / development using filtered MTBS, limited numbers of fires
# yr = '2020'
# fires_raw = ee.FeatureCollection('projects/pyregence-ee/assets/conus/mtbs/mtbs_fires_2020_20230406')
# irwin_list = [
#     '56C3E0AE-98EF-4433-863D-23A5DFF3D0C2', '60146AC0-3AC8-462B-AC80-60D31F1B4D96', 
#     '4CE00061-B897-49DC-8839-D08E040FF65D', 'BDE833A4-B278-4418-8526-67F2BFC7D7CD'] 
#     #BOBCAT = 56C3E0AE-98EF-4433-863D-23A5DFF3D0C2, DOME = 60146AC0-3AC8-462B-AC80-60D31F1B4D96, 
#     #CASTLE = 4CE00061-B897-49DC-8839-D08E040FF65D (note multipolygon), SILVERADO = BDE833A4-B278-4418-8526-67F2BFC7D7CD (late fire 2020-10-26, should be filtered out)
# irwin_list_failure = ['6F0ECD19-02D6-4987-A2A4-FEEDA94B35BD'] #09-28 fire
# fires = fires_raw.filter(ee.Filter.inList('irwinID',irwin_list_failure))
# data_origin = 'mtbs1'
# #output folder
# asset_folder = 'projects/pyregence-ee/assets/fires_bs_tool/dmn_testing'

# # ## OPTION Test / development using filtered MTBS, limited numbers of fires
# # Hunting 2019 Expanding window issue - pick 'expanding' and Sept run below
# yr = '2019'
# fires_raw = ee.FeatureCollection("projects/pyregence-ee/assets/conus/mtbs/mtbs_fires_2019_20230920") 
# #bad fire is AR3481609418020190917?
# fireBad = fires_raw.filter(ee.Filter.eq('Event_ID', 'AR3481609418020190917'))
# fires = fireBad
# data_origin = 'mtbsfireBad'
# #output folder
# asset_folder = 'projects/pyregence-ee/assets/fires_bs_tool/dmn_testing'


#Probably does not work with NIFC, unless various property names match. 
    # ### OPTION x: NIFC option
    # fires_asset = f"projects/pyregence-ee/assets/conus/nifc/nifc_fires_{yr}_gte100ac_20230406"
    # fires = ee.FeatureCollection(fires_asset)
    # data_origin = 'nifc'
    # asset_folder = 'projects/pyregence-ee/assets/conus/nifc'


## USER SETTINGS **********************************

export_flag = True #severity classes, composite/mosiac conus
per_fire_export = False #indiv fire results for dNBR and severity. #loop now, but dies on laptop. try to use fitoprincipe batch export?
csv_export_flag = True #csv log/diagnostic dates and info
#Variants of recent fire windows
# 'fixed' - fixed 90 days post fire, but won't go past sim date
# 'expanding' - variable days expanding from fire to current/sim date
# 'sliding' - fixed 90 day window, sliding along to most recent 90 days to current/sim date.  
recent_type = 'expanding' 

#Set simulation date (by index)
sim_index = 0 #pick 0 - 3 for sept, dec, march, june simulation
sim_date_list = [
    datetime.datetime(int(yr), 9, 30), 
    datetime.datetime(int(yr), 12, 31), 
    datetime.datetime((int(yr)+1), 3, 31), 
    datetime.datetime((int(yr)+1), 6, 30)]

# NOTE: Fires are filtered to the SAME date across all simulations
fire_filter_date = datetime.datetime(int(yr), 9, 20) 


##### SET (SIMULATED) RUN DATE #####

sim_date = sim_date_list[sim_index] 
sim_date_ee = ee.Date(sim_date)
#For consistency, foramt the sim date the same as Discovery date as from pull_mtbs_fires.py
sim_date_formatted = str(sim_date)[0:10]
print(f'Simulated run date: {sim_date_formatted}')

# NOTE: Fires are filtered to the SAME date across all simulations
filter_date = fire_filter_date
filter_date_ee = ee.Date(filter_date)
filter_date_formatted = str(filter_date)[0:10]
print(f'Fire filter date: {filter_date_formatted}')

# capture the code run date (today's / current date) as well
code_date = datetime.datetime.now()
code_date_formatted = str(code_date)[0:10]

# Map over the fires, adding the simulated run date 
#  note: must use subfunction set_windows_sim() in bs calc function
#  Also add id field for labeling bands in exported intermediate products
def set_sim_date(feat: ee.Feature):
    fire_date = ee.Date(feat.getString('Discovery'))
    #find out if fire is after filter date (negative is before filter date)
    days_from_filter = fire_date.difference(filter_date_ee,'day')
    #Also want to know discovery in reference to simulation date
    #negative days mean Discovery was BEFORE simulated date
    days_from_sim = fire_date.difference(sim_date_ee,'day')
    #Create a name for the individual fire export.
    #concatenate name pieces. Note: cannot start with number, if doing band-method. 
    fire_name_raw = ee.String(feat.get('Incid_Name'))
    # cannot have () and probably other special characters in band names, also removes all whitespace
    fire_name = fire_name_raw.replace('[^a-zA-Z0-9]', "", 'g')
    sep = ee.String('_')
    fire_event = ee.String(feat.get('Event_ID'))
    fire_id = fire_event.cat(sep).cat(fire_name)
    return feat.set('sim_date',sim_date_formatted, 
                    'days_from_sim',days_from_sim,
                    'filter_date',filter_date_formatted,
                    'days_from_filter',days_from_filter,
                    'fire_id',fire_id,
                    'code_run_date',code_date_formatted,
                    'recent_type',recent_type)
fires_dt = fires.map(set_sim_date)

# id_list = fires_dt.aggregate_array('fire_id')
# print(id_list.getInfo())

# Filter fires to only those that were discovered PRIOR to the filter date (plus a day padding for time ranges)
fires_sim = fires_dt.filter(ee.Filter.lt('days_from_filter', -1))

##### METADATA COLLECTION 1 #####
# Metadata collection and print
fires_count = fires.size().getInfo()
print(f'Total Fires in FeatureCollection: {fires_count}')
fires_sim_count = fires_sim.size().getInfo()
print(f'Fire count post simulated date filtering: {fires_sim_count}')

metadata = {
    'simulated_run_date':sim_date.strftime('%Y%m%d'),
    'code_run_date':code_date.strftime('%Y%m%d'),
    'fire_count_original':fires_count,
    'fire_filter_date':fire_filter_date.strftime('%Y%m%d'),
    'fire_count_filtered':fires_sim_count,
    'code_origin':'BS_Mapper_testing202308',
    'recent_type':recent_type
}
#print(metadata)


###### GET FIRE WINDOW DIAGNOSTICS ######

bs_windows = ee.FeatureCollection(fires_sim.map(af.bs_get_windows))

desc_windows = f'bs_simrun{yr}_{sim_date.strftime("%Y%m%d")}_{data_origin}_recent{recent_type}_windows_{code_date.strftime("%Y%m%d")}'
#wanted properties
wanted_cols = [
    'irwinID', 'Incid_Name', 'Event_ID', 'BurnBndAc',
    'filter_date', 'days_from_filter', 'Discovery', 
    'sim_date', 'days_from_sim', 
    'mode', 'recent_type', 'pre_start', 'pre_end', 'post_start', 'post_end',
    'code_run_date']
task_csv = ee.batch.Export.table.toDrive(
    collection=bs_windows.set('code_run_date', code_date),
    description=desc_windows,
    selectors=wanted_cols,
    folder='Fire',
    fileFormat='CSV')

if csv_export_flag == True:
    print(f'export task started: fire window diagnostics {desc_windows} to Google drive of user.')
    task_csv.start()


##### RUN BURN SEVERITY #####

# Returns RdNBR & BS for each fire ee.Feature
bs_coll_combined = ee.FeatureCollection(fires_sim).map(af.bs_calc_v2309)

# Separate out RdNBR and Miller Threshold severity classes
#bs_rdnbr = ee.ImageCollection(bs_coll_combined).select('RdNBR')
bs_coll = ee.ImageCollection(bs_coll_combined).select('MillersThresholds')


##### Export intermediate: individual features #####

if per_fire_export == True:

    #print('INDIVIDUAL FIRES NOT RUN: Need to write export.')

    desc_indiv = f'bs_simrun{yr}_{sim_date.strftime("%Y%m%d")}_{data_origin}_recent{recent_type}_perfire_{code_date.strftime("%Y%m%d")}'
    
    # TODO: batch export isn't picking up the path correctly??
    # Parameter "name" value "projects/pyregence-ee/assets" does not match the pattern "^projects/[^/]+/assets/.*$"
    
    path_coll = f'{asset_folder}/{desc_indiv}/'
    print(f'individual fire image collection: {path_coll}') 
    #fire_ids = bs_coll_combined.aggregate_array('fire_id')

    # TODO: Doesn't seem necessary in python geetools? 
    # use ! to submit a Command-Line call to the `earthengine` Command-Line Tool (https://developers.google.com/earth-engine/guides/command_line)
    #!earthengine create collection $path_coll

    #use geetools to batch export to an image collection
    geetools.batch.Export.imagecollection.toAsset(
        bs_coll_combined, 
        path_coll, 
        namePattern='{fire_id}', 
        scale=30,
        dataType="float", 
        region=None, 
        datePattern=None,
        extra=None, 
        verbose=False
    )

else:
    print('Skipping individual fire export')


###### METADATA COLLECTION 3 ######

#fire_calc_count = ee.ImageCollection(bs_coll_combined).size() # always an image, but some are no data
# Instead add up flag (0/1) counts in 'post_fire_calc'
fire_calc_count = ee.ImageCollection(bs_coll_combined).aggregate_sum('post_fire_calc')
metadata['fire_count_results'] = fire_calc_count

##### COMPOSITE BURN SEVERITY & FINAL EXPORT #####

# composite that ee.ImageCollection with a max() reducer, add metadata
bs_composite = bs_coll.max().rename('SEVERITY').set('Year',int(yr)).set(metadata)

# To Asset
desc = f'bs_simrun{yr}_{sim_date.strftime("%Y%m%d")}_{data_origin}_recent{recent_type}_{code_date.strftime("%Y%m%d")}'

utils.exportImgtoAsset(bs_composite, 
                    desc=desc,
                    region=None,
                    asset_folder=asset_folder, 
                    export_type='conus',
                    export=export_flag)



Simulated run date: 2020-09-30
Fire filter date: 2020-09-20
Total Fires in FeatureCollection: 814
Fire count post simulated date filtering: 722
export task started: fire window diagnostics bs_simrun2020_20200930_mtbs_recentexpanding_windows_20230922 to Google drive of user.
Skipping individual fire export
export task started: projects/pyregence-ee/assets/fires_bs_tool/dmn_testing/bs_simrun2020_20200930_mtbs_recentexpanding_20230922


# For one fire 
### Select a fire from a pre-existing fire featurecollection or provide your own fire feature asset (must have 'Discovery' property with value of format ee.String('yyyy-mm-dd') )


In [ ]:
# Running for a particular fire
# OUTDATED, using functions before simulation code & various fixes in there. 

# Select the fire based on irwinID (or name and edit filter below)
#fire_name = 'KNP COMPLEX' 
#fire_irwin = '499D6545-2A19-458A-8C7C-554932C6B86E'
#fire_name = 'WINDY' #there are 2 WINDY fires, using irwinID # fire_irwin = 'ACD5C207-BB83-4419-977D-3F64778081A3'
fire_irwin = 'ACD5C207-BB83-4419-977D-3F64778081A3'

# Read in fire data
fires = ee.FeatureCollection('projects/pyregence-ee/assets/conus/mtbs/mtbs_fires_2021_20230810')
print(f'Total Fires in FeatureCollection: {fires.size().getInfo()}')
#print(fires.first().getInfo()['properties'])
#print(fires.aggregate_array('Incid_Name').distinct().sort().getInfo())

# Filter to desired fire
#fires_f = fires.filter(ee.Filter.eq('Incid_Name',fire_name))
fires_f = fires.filter(ee.Filter.eq('irwinID',fire_irwin))
print(f'Filtered fire size: {fires_f.size().getInfo()}')

fire_name = fires_f.first().get('Incid_Name').getInfo().lower() #brittle, make sure only 1 feature
print(fire_name)

# Run burn severity tool 
bs_coll = ee.FeatureCollection(fires_f).map(af.bs_calc_new)
#bs_fire = af.bs_calc_new(fires_f.first()) #testing direct function call
#print(bs_fire.propertyNames().getInfo())
# while max() isn't really needed for 1 fire, keeping the pattern of the multi-fire runs
bs_composite = ee.ImageCollection(bs_coll).max().add(1).rename('SEVERITY') #shift severity values +1 so 0 can be nodata

# To Asset
# get current date to put into asset name
cur_date = datetime.datetime.now().strftime("%Y%m%d")
desc = f'mtbs_bs_{str(fire_name).replace(" ", "")}_{str(cur_date)}'
print(desc)
#fails with Image.clipToBoundsAndScale: The geometry for image clipping must be bounded.
utils.exportImgtoAsset(bs_composite,  
                    desc=desc,
                    region=bs_coll.geometry(),
                    asset_folder='projects/pyregence-ee/assets/fires_bs_tool/dmn_testing', 
                    export_type='single_fire', 
                    export=False) 


## RDNBR calculation only, as a test

In [ ]:
# Running for a particular fire - ONLY CALC RDNBR

# Select the fire based on irwinID 
    #fire_name = 'KNP COMPLEX' 
#fire_irwin = '499D6545-2A19-458A-8C7C-554932C6B86E'
    #fire_name = 'WINDY' #there are 2 WINDY fires, using irwinID # fire_irwin = 'ACD5C207-BB83-4419-977D-3F64778081A3'
fire_irwin = 'ACD5C207-BB83-4419-977D-3F64778081A3'

# Read in fire data
fires = ee.FeatureCollection('projects/pyregence-ee/assets/conus/mtbs/mtbs_fires_2021_20230810')
print(f'Total Fires in FeatureCollection: {fires.size().getInfo()}')
#print(fires.first().getInfo()['properties'])
#print(fires.aggregate_array('Incid_Name').distinct().sort().getInfo())

# Filter to desired fire
#fires_f = fires.filter(ee.Filter.eq('Incid_Name',fire_name))
fires_f = fires.filter(ee.Filter.eq('irwinID',fire_irwin))
print(f'Filtered fire size: {fires_f.size().getInfo()}')

fire_name = fires_f.first().get('Incid_Name').getInfo().lower() #brittle, make sure only 1 feature
print(fire_name)

# CalC RDNBR
rdnbr_coll = ee.FeatureCollection(fires_f).map(af.rdnbr_only_calc)
#bs_fire = af.bs_calc_new(fires_f.first()) #testing direct function call
#print(bs_fire.propertyNames().getInfo())
# while max() isn't really needed for 1 fire, keeping the pattern of the multi-fire runs
rdnbr_composite = ee.ImageCollection(rdnbr_coll).max().rename('RDNBR') 

# To Asset
# get current date to put into asset name
cur_date = datetime.datetime.now().strftime("%Y%m%d")
desc = f'mtbs_bs_rdnbr_{str(fire_name).replace(" ", "")}_{str(cur_date)}'
print(desc)
#fails with Image.clipToBoundsAndScale: The geometry for image clipping must be bounded.
utils.exportImgtoAsset(rdnbr_composite,  
                    desc=desc,
                    region=rdnbr_coll.geometry(),
                    asset_folder='projects/pyregence-ee/assets/fires_bs_tool/dmn_testing', 
                    export_type='single_fire', 
                    export=False)


# ee authentication block

In [ ]:
# import ee
# ee.Authenticate()